An example simulation can be run using the R scripts in this directory. These generate the required input from an inference muData object and create a simulated Perturb-seq experiment with injected effect sizes stored in an output muData object.

First we need to create the simulation input muData object. For this, we start with an inference muData object containing gene expression data and guide assignments. We add the perturbation effects for the target-gene pairs we want to simulate in the "pairs_to_test" table:

In [2]:
source("1_create_sim_mudata.R")

Loading required package: Matrix

Warning message:
“package ‘Matrix’ was built under R version 4.3.2”
Loading required package: S4Vectors

Warning message:
“package ‘S4Vectors’ was built under R version 4.3.2”
Loading required package: stats4

Loading required package: BiocGenerics

Warning message:
“package ‘BiocGenerics’ was built under R version 4.3.2”

Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following objects are ma

Next, we fit negative binomial distributions to the expression distribution of each gene. This is done using DESeq2's "estimateDispersions" functions. These distributions will alter be used to simulate scRNA-seq counts for genes in perturbed and unperturbed cells.

In [3]:
source("2_fit_negative_binomial.R")

Warning message:
“package ‘SummarizedExperiment’ was built under R version 4.3.2”
Loading required package: MatrixGenerics

Warning message:
“package ‘MatrixGenerics’ was built under R version 4.3.2”
Loading required package: matrixStats

Warning message:
“package ‘matrixStats’ was built under R version 4.3.2”

Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, 

Now we can run the actual simulation. This simulates scRNA-seq counts for all genes and cells in the muData object and injects the effect sizes specified in "pairs_to_test" for specified genes in cells that have a guide RNA targeting the specified perturbation target. In addition, this can add guide-to-guide variability to simulate efficiency differences of individual guides. The degree of guide variability can be controlled by the "guide_var" argument of the "simulate_pert_data" function (guide_var = 0: no guide efficieny variability).

In [4]:
source("3_simulate_count_data.R")

Warning message:
“sampleMap[['assay']] coerced with as.factor()”


We can now load the muData object with the simulated data into R. The count matrix in "gene" now contains the simulated scRNA-seq counts:

In [9]:
library(MuData)
mu <- readH5MU("output//gasperini_simulation_output.h5mu")
assay(mu@ExperimentList$gene)[1:10, 1:10]  # top left corner of simulated counts matrix

Warning message:
“sampleMap[['assay']] coerced with as.factor()”
  [[ suppressing 10 column names ‘GCTTGAATCGAATGCT-1_1B_1_SI-GA-F2’, ‘AGCTTGATCGAGAGCA-1_1A_2_SI-GA-E3’, ‘CCCAATCTCCTCAATT-1_1B_1_SI-GA-F2’ ... ]]



10 x 10 sparse Matrix of class "dgCMatrix"
                                   
ENSG00000008853 . . . . . . . . . .
ENSG00000104679 2 . 1 . . . . . . 2
ENSG00000104689 . 2 . 1 . 1 . . . .
ENSG00000120889 . . 1 2 . . . . . .
ENSG00000120896 . . . . . . . . . .
ENSG00000120913 . . . 1 . . . . 3 .
ENSG00000147439 2 1 . . . . . . 1 .
ENSG00000147454 7 1 9 1 . 4 8 9 4 4
ENSG00000147457 1 . . . . . . . . 1
ENSG00000158941 . . . 1 . . . . 1 .

The specified effect sizes for each perturbed target-gene pair are in "pairs_to_test"

In [11]:
data.frame(mu@metadata$pairs_to_test)

effect_size,gene_id,intended_target_name
<dbl>,<chr>,<chr>
0.5,ENSG00000136856,candidate_enh_3
0.5,ENSG00000136925,candidate_enh_2


Mean and dispersion estimates for each gene that were inferred using DESeq2 can be found in the gene-level metadata of the gene expression data:

In [14]:
head(rowData(mu@ExperimentList$gene), 3)

DataFrame with 3 rows and 7 columns
                     symbol    gene_chr gene_start  gene_end      mean
                <character> <character>  <numeric> <numeric> <numeric>
ENSG00000008853     RHOBTB2        chr8   22844930  22844931 0.0154662
ENSG00000104679      R3HCC1        chr8   23145421  23145422 0.5180354
ENSG00000104689   TNFRSF10A        chr8   23082573  23082574 0.1094613
                dispersion disp_outlier_deseq2
                 <numeric>           <logical>
ENSG00000008853 207.330548               FALSE
ENSG00000104679   0.245677               FALSE
ENSG00000104689  11.067130               FALSE

In the future, we will add the simulated guide efficiencies in the guide-level metadata of the guide data:

In [12]:
head(rowData(mu@ExperimentList$guide))

DataFrame with 6 rows and 5 columns
                       targeting intended_target_name intended_target_chr
                     <character>          <character>         <character>
ATGTAGAAGGAGACACCGGG        TRUE      ENSG00000012660                chr6
GCGCAGAGGCGGATGTAGAG        TRUE      ENSG00000012660                chr6
AGAGCGTCGGGATATCGGGG        TRUE      ENSG00000072274                chr3
CTCAGAGCGTCGGGATATCG        TRUE      ENSG00000072274                chr3
GCCCTGCTACCCACTTACAG        TRUE      ENSG00000113552                chr5
GGGAGGCGTCCGTGTAAGTG        TRUE      ENSG00000113552                chr5
                     intended_target_start intended_target_end
                                 <numeric>           <numeric>
ATGTAGAAGGAGACACCGGG              53213723            53213738
GCGCAGAGGCGGATGTAGAG              53213738            53213754
AGAGCGTCGGGATATCGGGG             195808960           195808975
CTCAGAGCGTCGGGATATCG             195808975           1958

For future development, we will also implement options to vary the number of cells per perturbation and baseline gene expression levels.